# Import des modules

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from ast import literal_eval
import csv
import re

# Création des variables

In [2]:
DF0 = pd.read_csv("./ReferentielPrenoms.csv",converters={"FormesNonNormalisées": literal_eval}).set_index("PrénomNormalisé")
DF0

,PrénomFrançais,Types,FormesNonNormalisées
PrénomNormalisé,,,
adalhelm,alleaume,"['masc', 'germanique']","[adalhelm, adelhelm, alhelm]"
adam,adam,"['masc', 'hébraïque']",[adam]
adelheit,adélaïde,"['fém', 'germanique']",[adelheit]
adolff,adolphe,"['masc', 'germanique']","[adolf, adolff]"
agnese,agnès,"['fém', 'grec']","[nese, agnese]"
...,...,...,...
wilhelm,guillaume,"['masc', 'germanique']",[wilhelm]
willibolt,NaN,"['masc', 'germanique']","[willebolt, willibolt]"
wirich,NaN,"['masc', 'germanique']","[wirich, wyrich]"


# Création de la soupe

In [3]:
soup = BeautifulSoup(open('./../../../Documents/base stras/aves_aa_195_p1.xml', 'r'))

# Création de la liste des éléments de la soupe à comparer

In [4]:
ListePrénomDoc = []
for forename in soup.find_all('forename'):
    forename = forename.get_text().lower().replace('\n', ' ')
    forename = re.sub(' +',' ',forename)
    ListePrénomDoc.append(forename)
print (ListePrénomDoc)

['johanns', 'niclas', 'hanns', 'heinrich', 'walther', 'bernhart', 'hanns', 'hannsheinrich', 'friedrich', 'hans hiltebrant', 'hannsheinrich', 'hanns', 'cüne', 'jerge', 'hanns', 'burckart', 'hanns', 'jacob', 'bernhart', 'hesseman', 'claus', 'hanns', 'niclaus', 'hanns cünrat', 'cünrat', 'ulrich', 'claus friderich', 'wolffhelm', 'walther', 'bernhart', 'hanns', 'peter', 'reimbolt', 'reimbolt', 'jerge', 'hanns', 'burckart', 'diebolt', 'claus', 'ludwig', 'reimbolt', 'hanns balthazar', 'hanns', 'hanns', 'claus', 'thoman', 'thoman', 'cüntze', 'jacob', 'hanns tietrich', 'hanns', 'boldel', 'peter', 'caspar', 'hanns', 'hanns', 'diebolt', 'thoman', 'bechtolt', 'hanns', 'claus', 'bechtolt', 'hanns', 'hanns', 'hanns', 'erbe', 'obreht', 'hanns', 'cunrat', 'hanns', 'steffan', 'hanns', 'reimbolt', 'hanns', 'heinrich', 'heinrich', 'heinrich', 'frantz', 'cüntzelin', 'heintzman', 'jacob', 'caspar', 'peter', 'hanns jerge', 'dietrich', 'hanns', 'hanns', 'hanns', 'peter', 'volmar', 'contz', 'henß', 'hanns', '

# Comparaison de la liste de la soupe à notre dataframe

In [5]:
for element in ListePrénomDoc :
    
    #Création des listes de références à partir du dataframe DF1.
    Colonne_FormesNonNormalisées = DF0['FormesNonNormalisées'].tolist()
    Colonne_PrénomNormalisé = DF0.index.tolist()
    
    #Vérification de l'existence de la forme non normalisée dans la liste des listes de formes non-normalisées.
    CheckForenameNN = any(element in sublist for sublist in Colonne_FormesNonNormalisées)
    DicoPersonne = {}

    #Sinon, on demande un str en entrée pour le comparer à la liiste des formes normalisées.
    if CheckForenameNN != True :
        ForenameN = input('Entrez la forme normalisée du prénom ' + element + ' :')
        
        #Si l'entrée figure dans la liste des prénoms normalisées, on ajoute la forme non normalisée à la liste de celles-ci dans le dataframe.
        if ForenameN in Colonne_PrénomNormalisé:
            CelluleModif = DF0.loc[ForenameN, 'FormesNonNormalisées']
            #print(type(CelluleModif))
            #print(CelluleModif)
            CelluleModif.append(element)
            
        #Sinon, on crée un dataframe qu'on additionnera à l'existant.
        else :
            
            #On crée le couple 'PrénomNormalisé' dont la valeur correspond à la forme la plus courante du prénom.
            DicoPersonne['PrénomNormalisé'] = ForenameN
            
            #On crée le couple 'FormeFR' dont la valeur correspond à la traduction française du prénom.
            FormeFR = input('Entrez la traduction française du prénom ' + element +' (possibilité de laisser vide)')
            DicoPersonne['PrénomFrançais'] = FormeFR
            
            #On crée le couple 'Types' dont la valeur est une liste contenant des tags proposant une description typologique du prénom.
            typologie = []
            n = int(input('Entrez le nombre de types à attribuer au prénom ' + element + ' :'))
            for i in range(0,n):
                Type = input('Entrez un des types à attribuer au prénom ' + element + ' :')
                typologie.append(Type)
            DicoPersonne['Types'] = typologie
            
            #On crée le couple 'FormesNonNormailglisées' dont la valeur est une liste contenant toutes les formes alternatives du prénom.
            ListeNN = []
            ListeNN.append(element)
            if element != ForenameN :
                ListeNN.append(ForenameN)
            DicoPersonne['FormesNonNormalisées'] = ListeNN
            
            #On crée le dataframe à ajouter à l'existant.
            DF1 = pd.DataFrame([DicoPersonne]).set_index('PrénomNormalisé')
            
            #On concatène le dictionnaire obtenu à notre Referentiel avant de recommencer toute l'opération.
            DF0 = pd.concat([DF0,DF1])
DF0

,PrénomFrançais,Types,FormesNonNormalisées
PrénomNormalisé,,,
adalhelm,alleaume,"['masc', 'germanique']","[adalhelm, adelhelm, alhelm]"
adam,adam,"['masc', 'hébraïque']",[adam]
adelheit,adélaïde,"['fém', 'germanique']",[adelheit]
adolff,adolphe,"['masc', 'germanique']","[adolf, adolff]"
agnese,agnès,"['fém', 'grec']","[nese, agnese]"
...,...,...,...
wilhelm,guillaume,"['masc', 'germanique']",[wilhelm]
willibolt,NaN,"['masc', 'germanique']","[willebolt, willibolt]"
wirich,NaN,"['masc', 'germanique']","[wirich, wyrich]"


In [7]:
DF0 = DF0.sort_values('PrénomNormalisé')
DF0
DF0.to_csv("./ReferentielPrenoms.csv")